In [3]:
import pandas as pd
import numpy as np
import regex as re
from matplotlib import pyplot as plt
import seaborn as sns
import nltk
from collections import Counter
from yellowbrick.text import PosTagVisualizer
import contractions
import os

In [ ]:
df = pd.read_csv("tweetsDataset.csv", encoding = "latin-1")
df.head()

In [6]:
df.columns = ["Sentiment Level", "Index", "Date", "No_Query", "Username", "Tweet"]
df.drop("No_Query", inplace = True, axis = 1)

In [18]:
df

,Tweet,Sentiment Level
0,is upset that he cant update his Facebook by t...,0
1,Kenichan I dived many times for the ball Manag...,0
2,my whole body feels itchy and like its on fire,0
3,nationwideclass no its not behaving at all im ...,0
4,Kwesidei not the whole crew,0
...,...,...
839994,Oficially done with drivers ed stuff Now I jus...,4
839995,jimhunt Thank YOU for sharing and caring Glad...,4
839996,buberzionist ok objection sustained,4
839997,is twittin while my baby girl sleeps,4


In [8]:
df_text = df["Tweet"]
df_class = df["Sentiment Level"]

In [9]:
print(df_text.shape)

(1599999,)


In [10]:
df = pd.concat([df_text, df_class], axis = 1)
df_negative = df.iloc[0:40000]
df_positive = df.loc[df["Sentiment Level"] == 4]
df_positive = df_positive.iloc[0:40000]

In [11]:
df = pd.concat([df_negative, df_positive], axis = 0)
df

,Tweet,Sentiment Level
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0
...,...,...
839994,Oficially done with drivers ed stuff. Now I ju...,4
839995,@jimhunt Thank YOU for sharing and caring! Gl...,4
839996,@buberzionist ok objection sustained,4
839997,is twittin' while my baby girl sleeps,4


In [12]:
df.shape

(80000, 2)

In [13]:
#removing punctuation 
df["Tweet"] = df["Tweet"].str.replace(r'[^\w\s]+', '')
df["Tweet"] = df["Tweet"].str.replace("^@[a-z0-9A-Z]+", '')


C:\Users\Chen Kang\AppData\Local\Temp\ipykernel_4840\2144257823.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet"] = df["Tweet"].str.replace(r'[^\w\s]+', '')
C:\Users\Chen Kang\AppData\Local\Temp\ipykernel_4840\2144257823.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet"] = df["Tweet"].str.replace("^@[a-z0-9A-Z]+", '')


In [14]:
emoj = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
    "]+", re.UNICODE)

df["Tweet"] = df["Tweet"].apply(lambda x : re.sub(emoj, "", x))
df["Tweet"] = df["Tweet"].apply(lambda x : x.encode("ascii", "ignore").decode())

df

,Tweet,Sentiment Level
0,is upset that he cant update his Facebook by t...,0
1,Kenichan I dived many times for the ball Manag...,0
2,my whole body feels itchy and like its on fire,0
3,nationwideclass no its not behaving at all im ...,0
4,Kwesidei not the whole crew,0
...,...,...
839994,Oficially done with drivers ed stuff Now I jus...,4
839995,jimhunt Thank YOU for sharing and caring Glad...,4
839996,buberzionist ok objection sustained,4
839997,is twittin while my baby girl sleeps,4


In [15]:
df.notna()

,Tweet,Sentiment Level
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
...,...,...
839994,True,True
839995,True,True
839996,True,True
839997,True,True


In [22]:
max = 0;

for rows in df["Tweet"]: 
    if len(rows) > max:
        max = len(rows)
        
print(max)

159


In [21]:
from nltk.tokenize import word_tokenize
df["Tweet_tokenize"] = df["Tweet"].apply(word_tokenize)

df.head()

,Tweet,Sentiment Level,Tweet_tokenize
0,is upset that he cant update his Facebook by t...,0,"[is, upset, that, he, cant, update, his, Faceb..."
1,Kenichan I dived many times for the ball Manag...,0,"[Kenichan, I, dived, many, times, for, the, ba..."
2,my whole body feels itchy and like its on fire,0,"[my, whole, body, feels, itchy, and, like, its..."
3,nationwideclass no its not behaving at all im ...,0,"[nationwideclass, no, its, not, behaving, at, ..."
4,Kwesidei not the whole crew,0,"[Kwesidei, not, the, whole, crew]"


In [23]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))



LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Chen Kang/nltk_data'
    - 'C:\\Users\\Chen Kang\\.conda\\envs\\TensorFlowCourse\\nltk_data'
    - 'C:\\Users\\Chen Kang\\.conda\\envs\\TensorFlowCourse\\share\\nltk_data'
    - 'C:\\Users\\Chen Kang\\.conda\\envs\\TensorFlowCourse\\lib\\nltk_data'
    - 'C:\\Users\\Chen Kang\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [25]:
import nltk 
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Chen
[nltk_data]     Kang\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True